In [5]:
import matplotlib.pyplot as plt
import networkx as nx
import random
from collections import Counter

In [6]:
def simulate_exploration(G, start, num_walks=30, max_steps=5):
    edge_hits = Counter()
    for _ in range(num_walks):
        current = start
        for _ in range(max_steps):
            neighbors = list(G.successors(current))
            if not neighbors:
                break
            next_node = random.choice(neighbors)
            edge_hits[(current, next_node)] += 1
            current = next_node
    return edge_hits

In [7]:
def plot_exploration_and_pruned(G, start, pruned_path, edge_hits, ax=None):
    if ax is None:
        fig, ax = plt.subplots(figsize=(8, 6))

    pos = nx.spring_layout(G, seed=42)

    # Draw all nodes lightly
    nx.draw_networkx_nodes(G, pos, node_size=500, node_color='lightgray', ax=ax)
    nx.draw_networkx_labels(G, pos, labels={n: G.nodes[n]['label'] for n in G.nodes}, font_size=8, ax=ax)

    # Base edges (exploration intensity)
    max_hits = max(edge_hits.values()) if edge_hits else 1
    for (u, v), hits in edge_hits.items():
        # Thickness proportional to exploration frequency
        width = 1 + (4 * hits / max_hits)
        ax.plot(
            [pos[u][0], pos[v][0]],
            [pos[u][1], pos[v][1]],
            linewidth=width,
            color='orange',
            alpha=0.4,
            zorder=1
        )

    # Draw pruned path on top
    path_edges = list(zip(pruned_path, pruned_path[1:]))
    nx.draw_networkx_edges(G, pos, edgelist=path_edges, width=4, edge_color='limegreen', ax=ax)
    nx.draw_networkx_nodes(G, pos, nodelist=pruned_path, node_size=600, node_color='limegreen', ax=ax)

    ax.set_title("Exploration (orange) → Pruned Path (green)")
    ax.axis('off')
    return ax